In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = 'train/',
    labels = 'inferred',
    label_mode = 'categorical',
    batch_size = 32,
    image_size = (96, 96))

Found 328 files belonging to 10 classes.


In [3]:
validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = 'validation/',
    labels = 'inferred',
    label_mode = 'categorical',
    batch_size = 32,
    image_size = (96, 96))

Found 86 files belonging to 10 classes.


In [4]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory = 'test/',
    labels = 'inferred',
    label_mode = 'categorical',
    batch_size = 32,
    image_size = (96, 96))

Found 86 files belonging to 10 classes.


In [24]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [25]:
data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(96, 96, 3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [26]:
from keras.models import Sequential
from keras.layers import Dense

num_classes = 10

model = Sequential([
    data_augmentation,
    keras.layers.experimental.preprocessing.Rescaling(1. / 255, input_shape=(96, 96, 3)),
  keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2,2)), 
  keras.layers.Conv2D(64, (3,3) , padding='same', activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2,2)),
  keras.layers.Conv2D(128, (3,3) , padding='same', activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2,2)),
  keras.layers.Dropout(0.25),  
  keras.layers.Flatten(),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.BatchNormalization(),
  keras.layers.Dropout(0.5),  
  keras.layers.Dense(num_classes, activation='softmax')
])

In [27]:
model.compile(optimizer='Adam', 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 96, 96, 3)         0         
_________________________________________________________________
rescaling_2 (Rescaling)      (None, 96, 96, 3)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 128)      

In [30]:
model.evaluate(test_ds)

3/3 [==============================] - 1s 57ms/step - loss: 0.5428 - accuracy: 0.8721


[0.5427522659301758, 0.8720930218696594]

In [31]:
model.save("model.h5")

In [5]:
from keras.models import load_model
# load model
model1 = load_model('model.h5')
# summarize model.
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_3 (Sequential)    (None, 96, 96, 3)         0         
_________________________________________________________________
rescaling_2 (Rescaling)      (None, 96, 96, 3)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 24, 24, 128)      

In [6]:
model1.evaluate(test_ds)

3/3 [==============================] - 1s 57ms/step - loss: 0.5428 - accuracy: 0.8721


[0.5427523255348206, 0.8720930218696594]